In [103]:
import os
import sys

import numpy as np
import pandas as pd

sys.path.insert(0, '..')

from app import db

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50)


<ipython-input-103-3ffd169b915e>:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [104]:
db_conn = db.get_db_conn()

In [105]:
query = """SELECT * FROM play_by_play"""
df_pbp = pd.read_sql(query, db_conn)
df_pbp.shape

(46189, 341)

In [106]:
df_pbp.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year
0,1,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,None,None,None,None,NaN,2020-09-13,900.0,1800.0,3600.0,Half1,0,NaN,0,1,NaN,0,15:00,ARI 35,0,NaN,GAME,None,NaN,0,0,NaN,0,0,0,None,None,NaN,NaN,None,None,None,NaN,None,None,3,3,NaN,None,None,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,...,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,0,None,None,NaN,NaN,NaN,NaN,2020,NaN,NaN,1,1,First down,1.0,16:25:00,None,Levi's Stadium,"Hazy Temp: 66° F, Humidity: 68%, Wind: 6 mph",10160000-0581-80a3-3e67-fadbeaac892d,0,0,GAME_START,0,None,None,None,1,Field goal,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,24,20,Home,-4,44,7.0,48.5,1,outdoors,None,None,None,Kyle Shanahan,Kliff Kingsbury,SFO01,Levi's Stadium,NaN,None,NaN,None,NaN,None,NaN,NaN,0.0,NaN,0,0,0,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2020
1,39,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,SF,home,ARI,ARI

## Fix the None passer value for qb spikes

In [107]:
df_pbp[df_pbp['play_type'] == 'qb_spike'][['passer', 'passer_player_name']].head()

,passer,passer_player_name
105,None,K.Murray
408,None,M.Stafford
1470,None,J.Burrow
1895,None,S.Darnold
1898,None,S.Darnold


In [108]:
df_pbp[df_pbp['passer'].isna() & ~df_pbp['passer_player_name'].isna()].head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year
105,2246,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,ARI,away,SF,SF,38.0,2020-09-13,3.0,3.0,1803.0,Half1,0,14.0,0,2,3.0,0,00:03,SF 38,5,37.0,(:03) 1-K.Murray spiked the ball to stop the clock.,qb_spike,0.0,0,0,0.0,0,1,0,None,None,0.0,NaN,None,None,None,NaN,None,None,2,0,0.0,None,None,0.0,2.0,13,7,7.0,13.0,-6.0,7.0,13.0,-6.0,0.633804,0.001972,0.000410,0.007249,0.328688,0.000478,0.027398,0.0,0.0,1.121325,0.410856,3.592107,-3.592107,-3.402992,3.402992,7.867161,-7.867161,NaN,NaN,0.0,0.0,1.057868,-1.057868,2.736733,-2.736733,1.630592,-1.630592,9.066285,-9.066285,0.296946,0.703054,0.703054,0.296946,0.007650,0.695404,0.304596,0.232380,0.767620,-0.103910,...,NaN,None,None,None,NaN,None,None,None,0.0,None,None,None,NaN,0,None,None,0.0,0.0,0.0,0.0,2020,NaN,NaN,32,0,Field goal,2246.0,16:25:00,21:49:31,Levi's Stadium,"Hazy Temp: 66° F, Humidity: 68%, Wind: 6 mph",10160000-0581-80a3-3e67-fadbeaac892d,35,0,PASS,0,None,00:02,SF 38,14,Field goal,None,6.0,0:30,2.0,0.0,1.0,2.0,2.0,0.0,KICKOFF,FIELD_GOAL,00:30,00:00,ARI 25,SF 38,2098.0,2291.0,24,20,Home,-4,44,7.0,48.5,1,outdoors,None,None,None,Kyle Shanahan,Kliff Kingsbury,SFO01,Levi's Stadium,1.0,None,NaN,None,NaN,None,Na

In [109]:
df_pbp['passer'] = np.where(
    df_pbp['passer'].isna() & ~df_pbp['passer_player_name'].isna(),
    df_pbp['passer_player_name'],
    df_pbp['passer']
)

In [110]:
df_pbp[df_pbp['passer'].isna() & ~df_pbp['passer_player_name'].isna()].head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year


In [111]:
df_pbp[df_pbp['play_type'] == 'qb_spike'][['passer', 'passer_player_name']].head()

,passer,passer_player_name
105,K.Murray,K.Murray
408,M.Stafford,M.Stafford
1470,J.Burrow,J.Burrow
1895,S.Darnold,S.Darnold
1898,S.Darnold,S.Darnold


In [112]:
df_pbp.shape

(46189, 341)

## Add position to receiving stats

In [113]:
df_roster = pd.read_sql("""SELECT * FROM roster""", db_conn)
df_roster.shape

(4609, 22)

In [114]:
df_roster.head()

,season,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,height,weight,college,high_school,gsis_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,update_dt,headshot_url
0,2020,ARI,C,C,52.0,Active,Mason Cole,Mason,Cole,1996-03-28,6-5,292.0,Michigan,East Lake (FL),00-0034785,3115972.0,53d25371-e3ce-4030-8d0a-82def5cdc600,31067.0,12795.0,45547.0,2021-01-08T10:21:48Z,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/3115972.png
1,2020,ARI,C,C,53.0,Active,Lamont Gaillard,Lamont,Gaillard,1996-02-08,6-3,305.0,Georgia,Pine Forest (NC),00-0035536,3128707.0,0c8b0581-9ed2-488b-bcaa-ef783261dfd1,32011.0,13678.0,46618.0,2021-01-08T10:21:48Z,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/3128707.png
2,2020,ARI,CB,NB,33.0,Active,Byron Murphy,Byron,Murphy,1998-01-18,5-11,190.0,Washington,Saguaro (AZ),00-0035236,4038999.0,c025b513-9431-4097-bc25-9777bf08f846,31865.0,13560.0,44027.0,2021-01-08T10:21:48Z,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/4038999.png
3,2020,ARI,CB,None,20.0,Active,Prince Amukamara,Prince,Amukamara,1989-06-06,6-0,204.0,Nebraska,Apollo (AZ),00-0027957,13975.0,f1879cfa-4c07-4140-9da0-c7ebe9af2dfd,24806.0,7509.0,6171.0,2021-01-08T10:21:48Z,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/13975.png
4,2020,ARI,CB,None,39.0,Injured Reserve,Jace Whittaker,Jace,Whittaker,1995-07-16,5-11,185.0,Arizona,Oceanside (CA),00-0036043,3821572.0,528bf5c9-1d23-40c3-adda-df21f8f0e2ab,33089.0,14793.0,50465.0,2021-01-08T10:21:48Z,https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/3821572.png


In [115]:
# Convert the pbp_ids to the correct format
import codecs
import numpy as np

def convert_to_gsis_id(pbp_id):
    """
    Convert new player id columns to old gsis id
    """
    if pbp_id is None:
        return None
    
    if type(pbp_id) == float:
        return pbp_id
        
    return codecs.decode(pbp_id[4:-8].replace('-',''),"hex").decode('utf-8')

In [116]:
df_pbp['gsis_id'] = df_pbp['receiver_id'].apply(convert_to_gsis_id)

In [117]:
sum(df_pbp['receiver_id'].notnull()), sum(df_pbp['gsis_id'].notnull()), 

(18264, 18264)

In [118]:
df_pbp.shape

(46189, 342)

In [119]:
cols = ['season', 'position', 'gsis_id']
df_roster_slim = df_roster[cols]
print(df_roster_slim.shape)

df_roster_slim = df_roster_slim.drop_duplicates()
print(df_roster_slim.shape)

(4609, 3)
(4439, 3)


In [89]:
df_roster_slim[df_roster_slim['position'].isna()].shape

(1, 3)

In [90]:
df_roster_slim[df_roster_slim['position'].isna()]

,season,position,gsis_id
100,2020,None,None


In [120]:
df_roster_slim = df_roster_slim[~df_roster_slim['position'].isna()]
df_roster_slim = df_roster_slim[~df_roster_slim['gsis_id'].isna()]
df_roster_slim.shape

(4414, 3)

In [121]:
df_pbp_enriched = df_pbp.merge(df_roster_slim, how='left',
                               left_on=['season', 'gsis_id'],
                               right_on=['season', 'gsis_id']
                              )
df_pbp.shape, df_pbp_enriched.shape

((46189, 342), (46189, 343))

In [95]:
df_pbp_enriched[
    df_pbp_enriched['gsis_id'].notnull() & df_pbp_enriched['position'].isna()
].shape

(0, 343)

In [96]:
df_pbp_enriched.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year,gsis_id,position
0,1,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,None,None,None,None,NaN,2020-09-13,900.0,1800.0,3600.0,Half1,0,NaN,0,1,NaN,0,15:00,ARI 35,0,NaN,GAME,None,NaN,0,0,NaN,0,0,0,None,None,NaN,NaN,None,None,None,NaN,None,None,3,3,NaN,None,None,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,...,None,None,NaN,None,None,None,NaN,None,None,None,NaN,0,None,None,NaN,NaN,NaN,NaN,2020,NaN,NaN,1,1,First down,1.0,16:25:00,None,Levi's Stadium,"Hazy Temp: 66° F, Humidity: 68%, Wind: 6 mph",10160000-0581-80a3-3e67-fadbeaac892d,0,0,GAME_START,0,None,None,None,1,Field goal,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,24,20,Home,-4,44,7.0,48.5,1,outdoors,None,None,None,Kyle Shanahan,Kliff Kingsbury,SFO01,Levi's Stadium,NaN,None,NaN,None,NaN,None,NaN,NaN,0.0,NaN,0,0,0,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2020,None,NaN
1,39,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,SF,home,ARI,ARI,35.0,2020-09-13,900.0,1800.0,3600.

In [102]:
df_pbp[df_pbp['play_type'] == 'qb_spike'].head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year,gsis_id
105,2246,2020_01_ARI_SF,2020091311,SF,ARI,REG,1,ARI,away,SF,SF,38.0,2020-09-13,3.0,3.0,1803.0,Half1,0,14.0,0,2,3.0,0,00:03,SF 38,5,37.0,(:03) 1-K.Murray spiked the ball to stop the clock.,qb_spike,0.0,0,0,0.0,0,1,0,None,None,0.0,NaN,None,None,None,NaN,None,None,2,0,0.0,None,None,0.0,2.0,13,7,7.0,13.0,-6.0,7.0,13.0,-6.0,0.633804,0.001972,0.000410,0.007249,0.328688,0.000478,0.027398,0.0,0.0,1.121325,0.410856,3.592107,-3.592107,-3.402992,3.402992,7.867161,-7.867161,NaN,NaN,0.0,0.0,1.057868,-1.057868,2.736733,-2.736733,1.630592,-1.630592,9.066285,-9.066285,0.296946,0.703054,0.703054,0.296946,0.007650,0.695404,0.304596,0.232380,0.767620,-0.103910,...,None,None,None,NaN,None,None,None,0.0,None,None,None,NaN,0,None,None,0.0,0.0,0.0,0.0,2020,NaN,NaN,32,0,Field goal,2246.0,16:25:00,21:49:31,Levi's Stadium,"Hazy Temp: 66° F, Humidity: 68%, Wind: 6 mph",10160000-0581-80a3-3e67-fadbeaac892d,35,0,PASS,0,None,00:02,SF 38,14,Field goal,None,6.0,0:30,2.0,0.0,1.0,2.0,2.0,0.0,KICKOFF,FIELD_GOAL,00:30,00:00,ARI 25,SF 38,2098.0,2291.0,24,20,Home,-4,44,7.0,48.5,1,outdoors,None,None,None,Kyle Shanahan,Kliff Kingsbury,SFO01,Levi's Stadium,1.0,K.Murray,NaN,None,NaN,None,NaN,0.0,0.0,0.0,0,

In [122]:
db_conn.close()